# Upgraded Sentiment Analysis for Torchtext IMDB Dataset
Based on https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb

In [1]:
import torch
from torch import nn
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

/student/rolwesg/.local/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/student/rolwesg/.local/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

/student/rolwesg/.local/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [4]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [5]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [6]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

/student/rolwesg/.local/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        #output = [sent len, batch size, hid dim * num direction]
        
        #hidden = [num layers * num directions, batch size, hid dim]
        
        #cell = [num layers * num directions, batch size, hid dim]
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        
        #hidden = [batch size, hid dim * num directions]
        
        return self.fc(hidden)

        

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
           EMBEDDING_DIM,
           HIDDEN_DIM,
           OUTPUT_DIM,
           N_LAYERS,
           BIDIRECTIONAL,
           DROPOUT,
           PAD_IDX)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'{count_parameters(model):,} trainable parameters')

4,810,857 trainable parameters


In [10]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [11]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9587,  0.2406, -0.0266,  ...,  0.6569, -0.0137,  0.5520],
        [-0.3028, -0.1248,  0.2981,  ..., -0.6897, -0.1911,  0.2448],
        [-0.2024, -0.4517,  0.4865,  ..., -0.4081,  0.6948,  0.4065]])

In [12]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9587,  0.2406, -0.0266,  ...,  0.6569, -0.0137,  0.5520],
        [-0.3028, -0.1248,  0.2981,  ..., -0.6897, -0.1911,  0.2448],
        [-0.2024, -0.4517,  0.4865,  ..., -0.4081,  0.6948,  0.4065]])


In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [14]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [15]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [16]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        print(batch)
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [19]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    

/student/rolwesg/.local/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)



[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 131x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 733x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 135x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 330x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 102x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 251x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 627x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 67x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 597x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 274x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 301x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 126x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 50x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 210x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 151x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 260x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 372x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 285x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 211x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 490x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU

...


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 154x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 180x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 227x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 539x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 320x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 419x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 337x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 448x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 92x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 162x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 132x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 175x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 200x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 218x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 1058x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GP


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 323x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 139x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 455x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 157x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU 0)]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.cuda.LongTensor of size 964x32 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 32 (GPU

Epoch: 05 | Epoch Time: 0m 58s
	Train Loss: 0.249 | Train Acc: 90.22%
	 Val. Loss: 0.285 |  Val. Acc: 88.83%


In [20]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.293 | Test Acc: 88.20%
